In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp

In [10]:
def animate_phase_map(
        qs=np.linspace(-1.0, 1.0, 300),
        ps=np.linspace(-1.0, 1.0, 300),
        energy_func=None,
        hamiltonian_rhs=None,  # function (t, y) -> [dq/dt, dp/dt]
        levels=None,
        contour_line_numbers=60,
        title="Phase Space Energy",
        q_spec_label="(position)",
        p_spec_label="(momentum)",
        t_vals=np.linspace(0, 10, 100),   # times for animation
        init_points=[(0.5, 0.0), (-0.5, 0.5)],  # list of (q0, p0)
        save_anim=False,
        anim_path='visualizations/phase_map_animation.mp4',
        cmap="viridis",
        linecolor="black",
        fps=15
    ):
    """
    Animate phase space energy contours over time with fixed color scale.
    energy_func must be callable: energy_func(Q, P, t).
    """
    Q, P = np.meshgrid(qs, ps)

    # Precompute global min/max across all times for fixed color scale
    H_min, H_max = np.inf, -np.inf
    for t in t_vals:
        H = energy_func(Q, P, t)
        H_min = min(H_min, np.min(H))
        H_max = max(H_max, np.max(H))

    if levels is None:
        levels = np.linspace(H_min, H_max, contour_line_numbers)

     # Integrate trajectories
    trajectories = []
    for q0, p0 in init_points:
        sol = solve_ivp(
            hamiltonian_rhs, 
            t_span=(t_vals[0], t_vals[-1]),
            y0=[q0, p0],
            t_eval=t_vals,
            rtol=1e-9, atol=1e-9
        )
        trajectories.append(sol.y)  # shape (2, len(t_vals))

    fig, ax = plt.subplots(figsize=(8, 6))
    H0 = energy_func(Q, P, t_vals[0])
    contourf = ax.contourf(Q, P, H0, levels=levels, cmap=cmap)
    cbar = fig.colorbar(contourf, ax=ax, label='Hamiltonian Energy $\\mathcal{H}$')
    ax.set_title(title + f"\nTime t = {t_vals[0]:.2f}")
    ax.set_xlabel('$q$ ' + q_spec_label)
    ax.set_ylabel('$p$ ' + p_spec_label)
    ax.grid(True)


    # Line + marker for each trajectory
    lines = []
    points = []
    for _ in init_points:
        (line,) = ax.plot([], [], lw=1.5)
        (point,) = ax.plot([], [], 'o', markersize=6)
        lines.append(line)
        points.append(point)

    def update(frame_idx):
        t = t_vals[frame_idx]
        H = energy_func(Q, P, t)

        # Remove previous contours
        for coll in ax.collections:
            coll.remove()

        # # Update trajectories
        # for traj, line, point in zip(trajectories, lines, points):
        #     q_traj, p_traj = traj
        #     line.set_data(q_traj[:frame_idx+1], p_traj[:frame_idx+1])
        #     point.set_data(q_traj[frame_idx], p_traj[frame_idx])

        cf = ax.contourf(Q, P, H, levels=levels, cmap=cmap)
        ax.set_title(title + f"\nTime t = {t:.2f}")
        return ax.collections #+ lines + points

    anim = FuncAnimation(fig, update, frames=len(t_vals), blit=False, interval=1000/fps)

    if save_anim:
        anim.save(anim_path, fps=fps, dpi=200)
        plt.close(fig)
    else:
        plt.show()

    return anim


In [ ]:
def H_pendulum(q, p, t):
    m, l, g = 1.0, 1.0, 9.81
    return p**2 / (2 * m * l**2) - m * g * l * np.cos(q)

def dynamics_pendulum(t, x):
    m, l, g = 1.0, 1.0, 9.81
    q, p = x
    dqdt = p / (m * l**2)
    dpdt = -m * g * l * np.sin(q)
    return [dqdt, dpdt]

def H_damped_pendulum(q, p, t):
    m, l, g = 1.0, 1.0, 9.81   
    F0 = 0.1
    gamma = 0.1
    return np.exp(-gamma*t) * (p**2 / (2 * m * l**2) - m * g * l * np.cos(q))

def dynamics_damped_pendulum(t, x):
    m, l, g = 1.0, 1.0, 9.81   
    gamma = 0.1
    q, p = x
    dqdt = p / (m * l**2)
    dpdt = -m * g * l * np.sin(q) - gamma * p / (m * l**2)
    return [dqdt, dpdt]

def H_windy_pendulum(q, p, t):
    m, l, g = 1.0, 1.0, 9.81   
    F0 = 0.1
    gamma = 0.1
    F_ext = F0 * t
    return np.exp(-gamma*t) * (p**2 / (2 * m * l**2) - m * g * l * np.cos(q) - q * F_ext)

def dynamics_windy_pendulum(t, x):
    m, l, g = 1.0, 1.0, 9.81   
    F0 = 0.1
    gamma = 0.1
    F_ext = F0 * t
    q, p = x
    dqdt = p / (m * l**2)
    dpdt = -m * g * l * np.sin(q) - gamma * p / (m * l**2) + F_ext
    return [dqdt, dpdt]

def H_conservative_spring(q, p, t):
    k, m = 1.0, 1.0
    return 0.5 * k * q**2 + 0.5 * p**2 / m

def dynamics_conservative_spring(t, x):
    k, m = 1.0, 1.0
    q, p = x
    dqdt = p / m
    dpdt = -k * q
    return [dqdt, dpdt]

def H_damped_spring(q, p, t):
    k, m = 1.0, 1.0
    gamma = 0.1
    return np.exp(-gamma*t) * (0.5 * k * q**2 + 0.5 * p**2 / m)

def dynamics_damped_spring(t, x):
    k, m = 1.0, 1.0
    gamma = 0.1
    q, p = x
    dqdt = p / m
    dpdt = -k * q - gamma * p / m
    return [dqdt, dpdt]

def H_forced_spring(q, p, t):
    k, m = 1.0, 1.0
    F0, omega = 1.0, 1.0
    gamma = 0.1
    F_ext = F0 * np.sin(omega * t) * np.sin(2 * omega * t)
    return np.exp(-gamma*t) * (0.5 * k * q**2 + 0.5 * p**2 / m - q * F_ext)

def dynamics_forced_spring(t, x):
    k, m = 1.0, 1.0
    F0, omega = 1.0, 1.0
    gamma = 0.1
    F_ext = F0 * np.sin(omega * t) * np.sin(2 * omega * t)
    q, p = x
    dqdt = p / m
    dpdt = -k * q - gamma * p / m + F_ext
    return [dqdt, dpdt]

def H_henon_heiles(q, p, t):
    q1, p1 = 1.0, 1.0
    q2, p2 = q, p
    a = 1.0
    H = 0.5 * (q1 ** 2 + q2 ** 2) +  0.5 * (p1 ** 2 + p2 ** 2) + a * (q1 * q1 * q2 - q2 ** 3 / 3.0)
    return H

def dynamics_henon_heiles(t, x):
    q1, p1 = 1.0, 1.0
    q2, p2 = x
    a = 1.0
    dq1dt = p1
    dp1dt = -q1 - 2 * a * q1 * q2
    dq2dt = p2
    dp2dt = -q2 - a * (q1 ** 2 - q2 ** 2)
    return [dq2dt, dp2dt]

def H_duffing_unforced(q, p, t):
    alpha, beta, m = -1.0, 1.0, 1.0
    gamma = 0.3
    return np.exp(-gamma*t) * (0.5 * alpha * q**2 + 0.25 * beta * q**4 + 0.5 * p**2 / m)

def dynamics_duffing_unforced(t, x):
    alpha, beta, m = -1.0, 1.0, 1.0
    gamma = 0.3
    q, p = x
    dqdt = p / m
    dpdt = -alpha * q - beta * q**3 - gamma * p / m
    return [dqdt, dpdt]

def H_duffing_chaotic(q, p, t):
    alpha, beta, m = -1.0, 1.0, 1.0
    f = 0.39
    omega = 1.4
    gamma = 0.1
    F_ext = f * np.sin(omega * t)
    return np.exp(-gamma*t) * (0.5 * alpha * q**2 + 0.25 * beta * q**4 + 0.5 * p**2 / m - q * F_ext)

def dynamics_duffing_chaotic(t, x):
    alpha, beta, m = -1.0, 1.0, 1.0
    f = 0.39
    omega = 1.4
    F_ext = f * np.sin(omega * t)
    gamma = 0.1
    q, p = x
    dqdt = p / m
    dpdt = -alpha * q - beta * q**3 - gamma * p / m + F_ext
    return [dqdt, dpdt]

In [12]:
anim = animate_phase_map(
    energy_func=H_pendulum,
    hamiltonian_rhs=dynamics_pendulum,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Conservative Pendulum",
    save_anim=True,
    anim_path='single_pendulum.mp4',
    cmap='coolwarm'
)

In [13]:
anim = animate_phase_map(
    energy_func=H_damped_pendulum,
    hamiltonian_rhs=dynamics_damped_pendulum,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Damped Pendulum",
    save_anim=True,
    anim_path='damped_pendulum.mp4',
    cmap='coolwarm'
)

In [14]:
anim = animate_phase_map(
    energy_func=H_windy_pendulum,
    hamiltonian_rhs=dynamics_windy_pendulum,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Windy Pendulum",
    save_anim=True,
    anim_path='windy_pendulum.mp4',
    cmap='coolwarm'
)

In [15]:
anim = animate_phase_map(
    energy_func=H_conservative_spring,
    hamiltonian_rhs=dynamics_conservative_spring,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Conservative Spring",
    save_anim=True,
    anim_path='conservative_spring.mp4',
    cmap='coolwarm'
)

In [16]:
anim = animate_phase_map(
    energy_func=H_damped_spring,
    hamiltonian_rhs=dynamics_damped_spring,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Damped Spring",
    save_anim=True,
    anim_path='damped_spring.mp4',
    cmap='coolwarm'
)

In [17]:
anim = animate_phase_map(
    energy_func=H_forced_spring,
    hamiltonian_rhs=dynamics_forced_spring,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Forced Spring",
    save_anim=True,
    anim_path='forced_spring.mp4',
    cmap='coolwarm'
)

In [19]:
anim = animate_phase_map(
    energy_func=H_duffing_unforced,
    hamiltonian_rhs=dynamics_duffing_unforced,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Unforced Duffing Oscillator",
    save_anim=True,
    anim_path='unforced_duffing.mp4',
    cmap='coolwarm'
)

In [20]:
anim = animate_phase_map(
    energy_func=H_duffing_chaotic,
    hamiltonian_rhs=dynamics_duffing_chaotic,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Forced Duffing Oscillator",
    save_anim=True,
    anim_path='chaotic_duffing.mp4',
    cmap='coolwarm'
)

In [21]:
anim = animate_phase_map(
    energy_func=H_henon_heiles,
    hamiltonian_rhs=dynamics_henon_heiles,
    t_vals=np.linspace(0, 10, 100),  # animate from t=0 to 20
    init_points=[(0.5, 0.0), (-0.5, 0.5)],
    title="Phase Space Energy for Henon-Heiles System",
    save_anim=True,
    anim_path='henon_heiles.mp4',
    cmap='coolwarm'
)